In [5]:
import torch
import json
from peft import LoraConfig,TaskType,get_peft_model
from datasets import Dataset
import itertools
import numpy as np
import pprint
from transformers import AutoTokenizer,AutoModelForCausalLM,Trainer,DataCollatorForSeq2Seq,TrainingArguments

In [6]:
# 加载模型和分词器
model_path="D:\Program Projects\Python Projects\DB-GPT\models\Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.half, device_map="auto")

# 这是模型决定的结束的词元token和它的id
base_model.config.pad_token_id=tokenizer.pad_token_id
print(f"eos_token: {tokenizer.eos_token}, id: {tokenizer.eos_token_id}") 
print(f"pad_token: {tokenizer.pad_token}, id: {tokenizer.pad_token_id}") 


eos_token: <|endoftext|>, id: 151643
pad_token: <|endoftext|>, id: 151643


In [3]:
# 加载训练数据
data=[]
data_path = "../huanhuan.json"
with open(data_path) as f:
    data = json.load(f)
data

[{'instruction': '小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  'input': '',
  'output': '嘘——都说许愿说破是不灵的。'},
 {'instruction': '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  'input': '',
  'output': '你们俩话太多了，我该和温太医要一剂药，好好治治你们。'},
 {'instruction': '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。',
  'input': '',
  'output': '出来走走，也是散心。'},
 {'instruction': '嬛妹妹，我虽是一介御医，俸禄微薄，可是我保证会一生一世对你好，疼爱你，保护你，永远事事以你为重。本来没半月一次到府上去请脉，能够偶尔见一次妹妹的笑靥，已经心满意足了，可谁知——而且我也知道，妹妹心里是不愿意去殿选的。',
  'input': '',
  'output': '实初哥哥这么说，就枉顾我们一直以来的兄妹情谊了，嬛儿没有哥哥，一直把你当作自己的亲哥哥一样看待，自然相信哥哥会待妹妹好的——自然了，以后有了嫂子，你也会对嫂子更好。'},
 {'instruction': '实初虽然唐突了妹妹，却是真心实意地希望妹妹不要去应选，这不仅仅是因为我心里一直把妹妹当成……其实更是因为甄伯父曾经救过家父的性命。',
  'input': '',
  'output': '我们两家是世交，昔年恩义不过是父亲随手之劳，不必挂怀。'},
 {'instruction': '可是我父亲当年被诬，起因也是因为后宫争斗，不能独善其身。一介御医尚且如此，何况妹妹如果被选中的话，会身在其中啊。',
  'input': '',
  'output': '实初哥哥的话我都明白，只是我不去应选，迟早也是玉娆，家中无子，女儿还能不孝吗？'},
 {'instruction': '在京里休息了这些日子，早已经调养过来了。',
  'input': '',
  'output': '如今你住在自己京城的宅子里，不比从前住在外祖家，一墙之隔，见面也方便。'},
 {'instruction': '是啊。可是我总还想着我们一起长

In [4]:
# 转换成Dataset
train_dataset = Dataset.from_list(data)
train_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3729
})

In [5]:
# 打印几条看看
data_temp = itertools.islice(train_dataset,5)
for i in data_temp:
    print(i)

{'instruction': '小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——', 'input': '', 'output': '嘘——都说许愿说破是不灵的。'}
{'instruction': '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。', 'input': '', 'output': '你们俩话太多了，我该和温太医要一剂药，好好治治你们。'}
{'instruction': '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。', 'input': '', 'output': '出来走走，也是散心。'}
{'instruction': '嬛妹妹，我虽是一介御医，俸禄微薄，可是我保证会一生一世对你好，疼爱你，保护你，永远事事以你为重。本来没半月一次到府上去请脉，能够偶尔见一次妹妹的笑靥，已经心满意足了，可谁知——而且我也知道，妹妹心里是不愿意去殿选的。', 'input': '', 'output': '实初哥哥这么说，就枉顾我们一直以来的兄妹情谊了，嬛儿没有哥哥，一直把你当作自己的亲哥哥一样看待，自然相信哥哥会待妹妹好的——自然了，以后有了嫂子，你也会对嫂子更好。'}
{'instruction': '实初虽然唐突了妹妹，却是真心实意地希望妹妹不要去应选，这不仅仅是因为我心里一直把妹妹当成……其实更是因为甄伯父曾经救过家父的性命。', 'input': '', 'output': '我们两家是世交，昔年恩义不过是父亲随手之劳，不必挂怀。'}


In [2]:
# 定义推理函数
def inference(model,tokenizer,text):
    model.eval() # 推理模式
    text = "\n".join(["<|system|>", "现在你要扮演皇帝身边的女人--甄嬛", "<|user|>", text + "<|assistant|>"]).strip()+ "\n"
    input_ids = tokenizer.encode(text,return_tensors='pt',truncation=True,max_length=1024)
    device = model.device
    generated_tokens=model.generate(
        input_ids=input_ids.to(device), 
        max_length=512, # 生成更过内容时，需要更多时间
        pad_token_id=tokenizer.pad_token_id,
        temperature=0.5,
        num_return_sequences=1
        
    )

    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens,skip_special_tokens=True)
    generated_text_answer=generated_text_with_prompt[0][len(text):]
    return generated_text_answer

In [7]:
text = "你会干什么"
pprint.pprint(inference(base_model,tokenizer,text))

d:\ProgramData\miniconda3\envs\datawhale\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
d:\ProgramData\miniconda3\envs\datawhale\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:580: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at .

'我会扮演甄嬛，扮演一个聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机

In [8]:
# 分词编码及预处理
def process_func(example):

    # 定义列表
    input_ids, labels = [], []


    # 设置编码最长编码
    MAX_LENGTH = 512 


    # instruction文本、编码
    text_instruction = "\n".join(["<|system|>", "现在你要扮演皇帝身边的女人--甄嬛", "<|user|>", example["instruction"][0] + example["input"][0] + "<|assistant|>"]).strip()+ "\n"
    instruction = tokenizer.encode(text=text_instruction,add_special_tokens=True, truncation=True, max_length=MAX_LENGTH)


    # response文本、编码
    text_response = example["output"][0]
    response = tokenizer.encode(text=text_response, add_special_tokens=False, truncation=True,max_length=MAX_LENGTH)
    


    # input_ids编码 = 指令 + 回复 + 结尾
    input_ids = instruction + response + [tokenizer.eos_token_id]

    
    # labels编码 = 填充（长度等于指令） + 回复 + 结尾
    labels = [tokenizer.pad_token_id] * len(instruction) + response + [tokenizer.eos_token_id]



    # 计算需要填充的长度512 - 64 = 448
    pad_len = MAX_LENGTH - len(input_ids) # 计算需要填充的长度
    

    # input_ids、labels 分别加上填充id，一起组成512长度的固定编码
    input_ids = input_ids + [tokenizer.pad_token_id] * pad_len
    labels = labels + [tokenizer.pad_token_id] * pad_len

    # 处理labels
    labels = [(l if l != tokenizer.pad_token_id else -100) for l in labels]
    
    # 编码返回到数据集
    example["input_ids"]=np.array([input_ids])
    example["labels"]=np.array([labels])
    
    return example


In [9]:
# map处理数据集
data=train_dataset.map(process_func,batched=True,batch_size=1,drop_last_batch=True)
split_dataset = data.train_test_split(test_size=0.1,shuffle=True,seed=123)
print(split_dataset)

Map: 100%|██████████| 3729/3729 [00:07<00:00, 503.91 examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'input_ids', 'labels'],
        num_rows: 3356
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'input_ids', 'labels'],
        num_rows: 373
    })
})


In [10]:
# peft微调
config = LoraConfig(
    init_lora_weights="olora"
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj","up_proj","o_proj","k_proj","down_proj","gate_proj","v_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1, # Dropout 比例
    bias='none'
)

lora_model = get_peft_model(base_model,config) # 生成了一个Lora模型
lora_model.print_trainable_parameters()
lora_model.save_pretrained("yuxin/qwen_2_0.5B_lora1")

trainable params: 4,399,104 || all params: 498,431,872 || trainable%: 0.8826


In [11]:
# 训练前推理,调用lora模型
lora_model.eval()
text = "你会干什么"
pprint.pprint(inference(lora_model,tokenizer,text)) # 结果与base_model一致，说明权重无变化

Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'我会扮演甄嬛，扮演一个聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机智、勇敢、善良、聪明、机

In [12]:
training_args = TrainingArguments(
    output_dir="yuxin/qwen_2_0.5B_output_dir",
    learning_rate=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=100,
    max_steps=100,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
)
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=lora_model,
    label_pad_token_id=-100,
    pad_to_multiple_of=None,
    padding=False
)



d:\ProgramData\miniconda3\envs\datawhale\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

trainer.train()
lora_model.save_pretrained("yuxin/qwen_2_0.5B_lora2")

max_steps is given, it will override any value given in num_train_epochs
100%|██████████| 100/100 [00:17<00:00,  5.80it/s]

{'train_runtime': 17.2537, 'train_samples_per_second': 5.796, 'train_steps_per_second': 5.796, 'train_loss': 4.386724853515625, 'epoch': 0.03}


In [14]:
# 训练后再推理
lora_model.eval()
text = "你会干什么"
pprint.pprint(inference(lora_model,tokenizer,text)) #结果不一致，说明经过训练后权重有变化

d:\ProgramData\miniconda3\envs\datawhale\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'我不会。皇上说，皇上是皇帝，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上。皇上是皇上，皇上是皇上，皇上是皇上，皇上是皇上，皇

In [15]:
# # 重新加载
# from peft import AutoPeftModelForCausalLM

# model = AutoPeftModelForCausalLM.from_pretrained("output_dir")
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model.eval()
# model = model.cuda()
# inputs = tokenizer("<|system|>\n现在你要扮演皇帝身边的女人--甄嬛\n<|user|>\n {}\n{}".format("你是谁？", "").strip() + "<|assistant|>\n", return_tensors="pt").to(model.device)
# # tokenizer.decode(model.generate(**inputs, max_length=128, do_sample=True)[0], skip_special_tokens=True)

# # peft官方
# outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=128)
# print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])


In [17]:
# 加载Lora 模型两种方式

# 方式一
# PeftModel类不能自动找到base_model位置，需要手动加载base_model，并导入到PeftModel类

from peft import PeftModel, PeftConfig

adapter_config = PeftConfig.from_pretrained("yuxin/qwen_2_0.5B_lora2") # 加载lora的config文件
base_model = AutoModelForCausalLM.from_pretrained(adapter_config.base_model_name_or_path) # 从config中加载找到base_model
lora_model = PeftModel.from_pretrained(base_model, "yuxin/qwen_2_0.5B_lora2") # 加载peft_config 和base_model

# 使用lora模型推理
text = "你在干什么"
answer= inference(lora_model,tokenizer,text)
answer


Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来

In [7]:
# 方式二
# AutoPeftModelForCausalLM类会根据lora_model文件中的config找到base_model
# 手动更改base_model的位置，则会报错

from peft import AutoPeftModelForCausalLM 

lora_model = AutoPeftModelForCausalLM.from_pretrained("yuxin/qwen_2_0.5B_lora2") 

# 使用lora模型推理
text = "你在干什么"
answer= inference(lora_model,tokenizer,text)
answer

d:\ProgramData\miniconda3\envs\datawhale\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来了？皇上怎么来

In [8]:
# 手动添加lora配置
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)
base_model.add_adapter(peft_config)